In [1]:
import pandas as pd 
import numpy as np 
import os
import re

In [ ]:
os.getcwd()

In [2]:
os.chdir("D:/cboobs/ApparelSales/ApparelSalesClassification")

In [3]:
pd.set_option('display.max_rows',10)

pd.set_option('display.max_columns',10)

df = pd.read_csv("data/IntlsalesReport.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37432 entries, 0 to 37431
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      37432 non-null  int64 
 1   DATE       37431 non-null  object
 2   Months     37407 non-null  object
 3   CUSTOMER   36392 non-null  object
 4   Style      36392 non-null  object
 5   SKU        34958 non-null  object
 6   Size       36392 non-null  object
 7   PCS        36392 non-null  object
 8   RATE       36392 non-null  object
 9   GROSS AMT  36392 non-null  object
dtypes: int64(1), object(9)
memory usage: 2.9+ MB


In [ ]:
# uniques of each column

for col in df.describe(include='object').columns:
    print('Column Name: ',col)
    print(df[col].unique())
    print('='*60 )

## Inspection

In [4]:
df1 = df.copy()
df1 = df1.drop(columns=['index'], axis=1)
# df1.drop_duplicates(keep='first', inplace=True)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37432 entries, 0 to 37431
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   DATE       37431 non-null  object
 1   Months     37407 non-null  object
 2   CUSTOMER   36392 non-null  object
 3   Style      36392 non-null  object
 4   SKU        34958 non-null  object
 5   Size       36392 non-null  object
 6   PCS        36392 non-null  object
 7   RATE       36392 non-null  object
 8   GROSS AMT  36392 non-null  object
dtypes: object(9)
memory usage: 2.6+ MB


In [5]:
# make all column names lowercase
df1.columns = [col.lower() for col in df1.columns]

# replace spaces with underscores in column names
df1.columns = df1.columns.str.lower().str.replace(' ', '_')

# check for the sum of all missing values in each column
df1.isna().sum()

date            1
months         25
customer     1040
style        1040
sku          2474
size         1040
pcs          1040
rate         1040
gross_amt    1040
dtype: int64

## Data Cleaning by column

### Date

In [6]:
# drop the rows where date is missing
df1.dropna(subset=['date'], inplace=True)



df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37431 entries, 0 to 37431
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       37431 non-null  object
 1   months     37407 non-null  object
 2   customer   36392 non-null  object
 3   style      36392 non-null  object
 4   sku        34958 non-null  object
 5   size       36392 non-null  object
 6   pcs        36392 non-null  object
 7   rate       36392 non-null  object
 8   gross_amt  36392 non-null  object
dtypes: object(9)
memory usage: 2.9+ MB


In [7]:
# convert the date column to datetime format
df1['date'] = pd.to_datetime(df1['date'], format='%m-%d-%y')

ValueError: time data "SKU" doesn't match format "%m-%d-%y", at position 162. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [ ]:
# check unique values in date column

df1.date.unique()

In [8]:
# output lists of unique values in date column - with numbers and without numbers

with_numbers = []
without_numbers = []

for x in df1['date'].unique():
    # Check if value contains any digit
    if any(char.isdigit() for char in str(x)):
        with_numbers.append(x)
    else:
        without_numbers.append(x)

print("Dates with numbers:")
print(with_numbers)
print("\n\n\nDates without numbers:")
print(without_numbers)

Dates with numbers:
['06-05-21', '06-08-21', '06-11-21', '06-15-21', '06-17-21', '06-18-21', '06-19-21', '06-23-21', '06-24-21', '06-28-21', '06-29-21', '07-01-21', '07-08-21', '07-09-21', '07-10-21', '07-12-21', '07-13-21', '07-16-21', '07-22-21', '07-24-21', '07-26-21', '07-28-21', '07-29-21', '07-30-21', '07-31-21', '08-04-21', '08-05-21', '08-06-21', '08-07-21', '08-10-21', '08-14-21', '08-15-21', '08-16-21', '08-19-21', '08-20-21', '08-21-21', '08-25-21', '08-26-21', '09-03-21', '09-06-21', '09-07-21', '09-09-21', '09-11-21', '09-14-21', '09-15-21', '09-16-21', '09-18-21', '09-21-21', '09-22-21', '09-24-21', '09-27-21', '09-29-21', '10-01-21', '10-02-21', '10-04-21', '10-05-21', '10-06-21', '10-08-21', '10-09-21', '10-11-21', '10-12-21', '10-13-21', '10-14-21', '10-15-21', '10-18-21', '10-20-21', '10-21-21', '10-23-21', '10-26-21', '10-29-21', '11-01-21', '11-03-21', '11-12-21', '11-16-21', '11-19-21', '11-22-21', '11-23-21', '11-26-21', '11-27-21', '11-29-21', '12-02-21', '12-06-

- From the above we can see that for date entries, names have been provided and in other instances styles
- We going to use these names and styles to fill some missing data in the customer and style columns.

In [9]:
# fill style with date value for with_numbers where date starts with a letter and style is null
for val in with_numbers:
    if isinstance(val, str) and val and val[0].isalpha():
        mask = (df1['date'] == val) & (df1['style'].isna())
        df1.loc[mask, 'style'] = val

# fill customer with date value for without_numbers where customer is null
for val in without_numbers:
    mask = (df1['date'] == val) & (df1['customer'].isna())
    df1.loc[mask, 'customer'] = val

# for 'SKU', 'Style', 'CUSTOMER' in date, fill customer with 'Unspecified'
specials = ['SKU', 'Style', 'CUSTOMER']
for special in specials:
    mask = (df1['date'] == special)
    df1.loc[mask, 'customer'] = 'Unspecified'

In [ ]:
df1.info()

##### Observation

- 2 observations from customer column were filled.
- 1,037 observations from style column were filled.

### months 

In [ ]:
# for the 24 null observations of months, inspect df1 to find out the 
# corresponding observations in other features


df1[df1.months.isna()]

- from the above we notice that there is no meaningful data from above therefore i will drop all observations where months is null

In [10]:
# drop nulls as a subset of months

df1.dropna(subset=['months'], inplace=True)
df1.months.unique()

array(['Jun-21', 'Jul-21', 'Aug-21', 'Sep-21', 'Oct-21', 'Nov-21',
       'Dec-21', 'Jan-22', 'Feb-22', 'Mar-22', 'Apr-22', 'May-22',
       'Stock', '43.00', '55.00', '53.00', '34.00', '119.00', '100.00',
       '152.00', '37.00', '114.00', '73.00', '128.00', '144.00', '45.00',
       '97.00', '311.00', '129.00', '8.00', '40.00', '127.00', '66.00',
       '15.00', '9.00', '147.00', '11.00', '89.00', '10.00', '38.00',
       '231.00', '25.00', '24.00', '289.00', '108.00', '49.00', '329.00',
       '16.00', '212.00', '39.00', '90.00', '206.00', '194.00', '491.00',
       '59.00', '104.00', '116.00', '4.00', '6171.00', '1486.00',
       '175.00', '1600.00', '52.00', '32.00', '30.00', '81.00', '20.00',
       '18.00', '31.00', '19.00', '23.00', '1122.00', '316.00', '117.00',
       '84.00', '164.00', '429.00', '399.00', '188.00', '192.00',
       '163.00', '106.00', '143.00', '214.00', '179.00', '2064.00',
       '17.00', '13.00', '307.00', '46.00', '487.00', '82.00', '109.00',
       '17

In [ ]:
df1.customer.unique()

In [ ]:
df1.tail(10)

In [11]:
# function to match formats like 'Jun-21'
def rformat(val):
    
    return bool(re.match(r'^[A-Za-z]{3}-\d{2}$', str(val)))

# function to match formats like '03-09-22'
def wformat(val):
    
    return bool(re.match(r'^\d{2}-\d{2}-\d{2}$', str(val)))

for idx, row in df1.iterrows():
    months_val = row['months']
    if not (rformat(months_val) or wformat(months_val)):
        customer_val = row['customer']
        date_val = row['date']
        if rformat(customer_val):
            df1.at[idx, 'months'] = customer_val
        elif wformat(date_val):
            df1.at[idx, 'months'] = date_val
        # else: pass (do nothing)


In [ ]:
df1.months.unique()

In [12]:
# Create a new column months_new from months
month_map = {
    '01': 'Jan', '02': 'Feb', '03': 'Mar', '04': 'Apr', '05': 'May',
    '06': 'Jun', '07': 'Jul', '08': 'Aug', '09': 'Sep', '10': 'Oct', '11': 'Nov', '12': 'Dec'
}

def map_month(val):
    s = str(val)
    if re.match(r'^\d{2}-\d{2}-\d{2}$', s):
        mm = s[:2]
        if mm in month_map:
            return month_map[mm] + s[2:]
    return val

df1['months'] = df1['months'].apply(map_month)

In [ ]:
df1.months.unique()

In [13]:

def remove_3_7(val):
    s = str(val)
    # match formats like 'Nov-03-21'
    if re.match(r'^[A-Za-z]{3}-\d{2}-\d{2}$', s):
        return s[:3] + s[6:]
    return val

df1['months'] = df1['months'].apply(remove_3_7)

In [14]:
def monthsCount(val):
    return bool(re.match(r'^[A-Za-z]{3}-\d{2}$', str(val)))

mnths = df1['months'].apply(monthsCount).sum()
nmbrs = len(df1) - mnths

print(f"Count with 'Mon-YY' format (e.g., Apr-22): {mnths}")
print(f"Count without 'Mon-YY' format: {nmbrs}")

Count with 'Mon-YY' format (e.g., Apr-22): 36391
Count without 'Mon-YY' format: 1016


##### Deductions

- There are 36391 observations with the right values. 
- 1016 have formats that can not be transformed to the needed format. 


##### Next Step

- Drop rows for the 1016

In [ ]:
df1.info()

In [15]:
def monthEdit(val):
    return bool(re.match(r'^[A-Za-z]{3}-\d{2}$', str(val)))

mask = df1['months'].apply(monthEdit)
df1 = df1[mask].reset_index(drop=True)

df1.info()
print('---'*60)
df1.months.unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36391 entries, 0 to 36390
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       36391 non-null  object
 1   months     36391 non-null  object
 2   customer   36391 non-null  object
 3   style      36391 non-null  object
 4   sku        34957 non-null  object
 5   size       36391 non-null  object
 6   pcs        36391 non-null  object
 7   rate       36391 non-null  object
 8   gross_amt  36391 non-null  object
dtypes: object(9)
memory usage: 2.5+ MB
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


array(['Jun-21', 'Jul-21', 'Aug-21', 'Sep-21', 'Oct-21', 'Nov-21',
       'Dec-21', 'Jan-22', 'Feb-22', 'Mar-22', 'Apr-22', 'May-22'],
      dtype=object)

- after dropping the 1016 rows, only sku remains with missing data

### customer

In [ ]:
df1.customer.unique()

In [16]:
# replace date-like values in customer with 'Unspecified'

dates_to_replace = ['Jun-21','Jul-21', 'Aug-21', 'Sep-21', 'Oct-21', 'Nov-21', 'Dec-21','Jan-22', 'Feb-22', 'Mar-22']
df1['customer'] = df1['customer'].replace(dates_to_replace, 'Unspecified')

df1.customer.unique()

array(['REVATHY LOGANATHAN', 'FARIA ESSOPP', 'MANGALAM SHOP',
       'THANA NAGISSWARY L MARIMUTHU', 'MR.ALWAR MURALI',
       'RAZIA ROSEANE NASER', 'SIRI PADALA', 'FUSION FASHIONS CORP.',
       'MIZNA WAHEEDH', 'AMANI CONCEPT TRADING LLC (KAPDA)', 'NITHARSHA',
       'SYEDA MORSHED', 'VINTAGE INDIA NYC', 'M/S CHARISMAKURTIES(DUBAI)',
       'MULBERRIES BOUTIQUE', 'COTTON CLOSET LTD', 'THANA MARIMUTHU',
       'SHWETA', 'MONIYSHAA', 'AANCHOL', 'KHUSBOO BEEHARRY', 'ARUNA DEVI',
       'YAMUNAH PUSPANATHAN', 'RISHIKESH DASHPUTRE', 'REGA',
       'SURE FASHIONS LLC', 'MR ALWAR MURALI', 'KOGILA SELLAPPAN',
       'MONISYAA', 'VINODHA PUSPANATHAN', 'REKA RASAKODY', 'ARKH FASHION',
       'PUVANES GANNASIN', 'VINI’S AUTHENTIC INDIAN ETHNIC WEAR',
       'BINCY SKARIA', 'YASHWINI REDDY', 'BANUJA RASAKODY',
       'VAHLAARMATHY', 'AKASH KAUSHAL', 'KIRUTHIKA V SURESHBABU',
       'RIVAAN LLC', 'SINDHU', 'DHENOOGA RAVINTHERAN',
       'VAHARSHA BOUTIQUE', 'MANISH DHOORUNDHUR', 'RINO SANDARAN',

In [17]:
# delete leading and trailing spaces

df1.customer = df1.customer.str.strip()
df1.customer.unique()
print(f"There are {df1.customer.nunique()} unique Customers")

There are 158 unique Customers


In [18]:
# make all observations in the column lowercase

df1.customer = [x.lower() for x in df1.customer]
df1.customer = df1.customer.astype("str")

print(df1.customer.unique())
print("==="*30)
print("==="*30)
print(f"There are {df1.customer.nunique()} unique Customers")

['revathy loganathan' 'faria essopp' 'mangalam shop'
 'thana nagisswary l marimuthu' 'mr.alwar murali' 'razia roseane naser'
 'siri padala' 'fusion fashions corp.' 'mizna waheedh'
 'amani concept trading llc (kapda)' 'nitharsha' 'syeda morshed'
 'vintage india nyc' 'm/s charismakurties(dubai)' 'mulberries boutique'
 'cotton closet ltd' 'thana marimuthu' 'shweta' 'moniyshaa' 'aanchol'
 'khusboo beeharry' 'aruna devi' 'yamunah puspanathan'
 'rishikesh dashputre' 'rega' 'sure fashions llc' 'mr alwar murali'
 'kogila sellappan' 'monisyaa' 'vinodha puspanathan' 'reka rasakody'
 'arkh fashion' 'puvanes gannasin' 'vini’s authentic indian ethnic wear'
 'bincy skaria' 'yashwini reddy' 'banuja rasakody' 'vahlaarmathy'
 'akash kaushal' 'kiruthika v sureshbabu' 'rivaan llc' 'sindhu'
 'dhenooga ravintheran' 'vaharsha boutique' 'manish dhoorundhur'
 'rino sandaran' 'vendan' 'vishal darshan boutique' 'kaveenaz collection'
 'alameloo(mala)' 'murugesan chandra' 'meera rasadurai' 'manjulika gupta'
 'gal

In [19]:
# Replace 'unspecified' in customer with corresponding date value if date has no numbers
for idx, row in df1.iterrows():
    if row['customer'] == 'unspecified':
        date_val = str(row['date'])
        # Check if date_val has no numbers
        if not any(char.isdigit() for char in date_val):
            df1.at[idx, 'customer'] = date_val.strip().lower()
        # else: pass (leave as 'unspecified')

- this was to replace the initial "unspecified" customers with the real customers from the date column.

##### Observation

- the number of unique customers reduced from 158 to 150

In [ ]:
df1.info()

### style

In [ ]:
# pd.set_option('display.max_rows',None)


df1['style'].value_counts()

print("===" *50)

print(f"There are {df1.customer.nunique()} unique styles")



In [20]:
df1['style'] = [x.lower() for x in df1['style']]
df1['style'] = df1['style'].str.strip()
print(f"There are {df1.customer.nunique()} unique styles")

There are 149 unique styles


In [ ]:
df1['style'].value_counts()

### sku

In [21]:
# Clean sku column: lowercase and strip whitespace

df1['sku'] = df1['sku'].astype(str).str.lower().str.strip()

# after converting sku to str, all NaN values will take an observation nan
df1.sku.value_counts()

sku
nan               1434
shipping           508
set268-kr-np-l      62
jne3555-kr-l        58
jne3548-kr-m        54
                  ... 
j0399-dr-l           2
j0092-set-m          2
j0238-lcd-m          2
jne3817-kr-xl        2
tag printing         1
Name: count, Length: 4598, dtype: int64

In [ ]:
df1.info()

- to replace nan with the right values, i will use observations from style and size.

### size 

In [22]:
df1['size'].unique()

array(['L', 'XL', 'XXL', 'S', 'M', 'Free', 'XS', 'XXXL', '5XL', '6XL',
       '4XL', 'FREE', 'S TO XXL', 's', 'm', 'l', 'xl', 'xxl', 'xxxl',
       '1.00', '2.00', '6.00', '4.00', '1000.00', '600.00', '3.00',
       '5.00', '15.00', '7.00', '8.00', '12.00', '11.00', '9.00',
       '698.00', '10.00'], dtype=object)

In [ ]:
df2 = df1.copy()

'''df3 = df1.copy()

df4 = df1.copy()'''

In [24]:
df2['skuuuu'] = df2['sku'].str.split('-').str[-1]
df2['skuuuu'].unique()

array(['l', 'xl', 'xxl', 's', 'm', 'kurti', 'xs', 'xxxl', 'xs.', 'm.',
       'l.', 's.', '74black', '74beige', '74pink', '74red', '62black',
       '164gold', '61black', 'nan', 'xl.', 'xxxl.', '83red', '62pink',
       'bl014', '71red', '187gold', '87black', '78beige', 'sar086',
       'sar079', 'sar043', 'sar044', 'sar045', 'sar046', 'sar033', '5xl',
       '6xl', '4xl', 'saree', 'sar070', 'jne3826', 'jne3827', 'jne3828',
       'jne3853', 'jne3855', 'jne3858', 'jne3829', 'jne3830', 'jne3849',
       'jne3852', 'jne3848', 'jne3831', 'jne3851', 'jne3832', 'jne3850',
       'jne3839', 'jne3840', 'jne3844', 'jne3845', 'jne3846', 'jne3847',
       'sar050', 'sar049', 'sc', 'sar031', 'sar027', 'sar038', 'free',
       'jne3843', 'sar054', 'sar029', 'jne3857', 's to xxl', 'shipping',
       'tag printing', 'tags(labour)', 'tags', 'label charge',
       'shipping charges', 'label manuf.chrage'], dtype=object)

In [26]:

df2['skuuuu'] = df2['skuuuu'].str.strip(".").replace(['5xl','6xl', '4xl'], ['xxxxxl', 'xxxxxxl', 'xxxxl'])
df2['skuuuu'].unique()

array(['l', 'xl', 'xxl', 's', 'm', 'kurti', 'xs', 'xxxl', '74black',
       '74beige', '74pink', '74red', '62black', '164gold', '61black',
       'nan', '83red', '62pink', 'bl014', '71red', '187gold', '87black',
       '78beige', 'sar086', 'sar079', 'sar043', 'sar044', 'sar045',
       'sar046', 'sar033', 'xxxxxl', 'xxxxxxl', 'xxxxl', 'saree',
       'sar070', 'jne3826', 'jne3827', 'jne3828', 'jne3853', 'jne3855',
       'jne3858', 'jne3829', 'jne3830', 'jne3849', 'jne3852', 'jne3848',
       'jne3831', 'jne3851', 'jne3832', 'jne3850', 'jne3839', 'jne3840',
       'jne3844', 'jne3845', 'jne3846', 'jne3847', 'sar050', 'sar049',
       'sc', 'sar031', 'sar027', 'sar038', 'free', 'jne3843', 'sar054',
       'sar029', 'jne3857', 's to xxl', 'shipping', 'tag printing',
       'tags(labour)', 'tags', 'label charge', 'shipping charges',
       'label manuf.chrage'], dtype=object)

In [27]:
# replace some values and make all lowercase

df2['size'] = df2['size'].replace(['5XL','6XL','4XL'], ['xxxxxl', 'xxxxxxl', 'xxxxl'])

df2['size'] = [x.lower() for x in df2['size']]

df2['size'].unique()

array(['l', 'xl', 'xxl', 's', 'm', 'free', 'xs', 'xxxl', 'xxxxxl',
       'xxxxxxl', 'xxxxl', 's to xxl', '1.00', '2.00', '6.00', '4.00',
       '1000.00', '600.00', '3.00', '5.00', '15.00', '7.00', '8.00',
       '12.00', '11.00', '9.00', '698.00', '10.00'], dtype=object)

In [45]:
# replace size in df2 for specific numeric values if skuuuu is in allowed set

sizes_to_replace = ['1.00', '2.00', '6.00', '4.00', '1000.00', '600.00', '3.00', '5.00', '15.00', '7.00', '8.00', '12.00', '11.00', '9.00', '698.00', '10.00']
allowed_sizes = ['l', 'xl', 'xxl', 's', 'm', 'xs', 'xxxl', 's to xxl', 'free', 'xxxxxl', 'xxxxxxl', 'xxxxl']
for idx, row in df2.iterrows():
    if row['size'] in sizes_to_replace:
        skuuuu_val = str(row['skuuuu']).strip().lower()
        if skuuuu_val in allowed_sizes:
            df2.at[idx, 'size'] = skuuuu_val
        
df2['size'].unique()

array(['l', 'xl', 'xxl', 's', 'm', 'free', 'xs', 'xxxl', 'xxxxxl',
       'xxxxxxl', 'xxxxl', 's to xxl', '1.00', '1000.00', '600.00',
       '2.00', '698.00', '4.00', '5.00', '15.00', '10.00'], dtype=object)

- after this, the sizes reduced from 20 t0 21 

In [29]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36391 entries, 0 to 36390
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       36391 non-null  object
 1   months     36391 non-null  object
 2   customer   36391 non-null  object
 3   style      36391 non-null  object
 4   sku        36391 non-null  object
 5   size       36391 non-null  object
 6   pcs        36391 non-null  object
 7   rate       36391 non-null  object
 8   gross_amt  36391 non-null  object
 9   skuuuu     36391 non-null  object
dtypes: object(10)
memory usage: 2.8+ MB


### pcs, rate, gross_amt

In [32]:
df2.pcs = df2.pcs.astype('float64')

In [33]:
df2.rate = df2.rate.astype('float64')

In [34]:
df2.gross_amt = df2.gross_amt.astype('float64')

In [35]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36391 entries, 0 to 36390
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       36391 non-null  object 
 1   months     36391 non-null  object 
 2   customer   36391 non-null  object 
 3   style      36391 non-null  object 
 4   sku        36391 non-null  object 
 5   size       36391 non-null  object 
 6   pcs        36391 non-null  float64
 7   rate       36391 non-null  float64
 8   gross_amt  36391 non-null  float64
 9   skuuuu     36391 non-null  object 
dtypes: float64(3), object(7)
memory usage: 2.8+ MB


### sku 

In [46]:
df3 = df2.copy()

df4 = df2.copy()

In [51]:
pd.set_option('display.max_rows',None)

df2.sku.value_counts()

sku
nan                           1434
shipping                       508
set268-kr-np-l                  62
jne3555-kr-l                    58
jne3548-kr-m                    54
jne3440-kr-l                    52
jne3548-kr-xl                   50
set268-kr-np-m                  50
jne3555-kr-m                    50
jne3555-kr-xl                   48
jne3405-kr-m                    46
set268-kr-np-xl                 46
set252-kr-pp-s.                 46
jne3555-kr-xxl                  44
jne2100-kr-144-m                44
jne3489-kr-xl                   42
jne3440-kr-xl                   42
jne3548-kr-l                    42
jne3489-kr-m                    42
jne3567-kr-m                    40
set286-kr-np-xl                 40
jne3555-kr-s.                   40
jne3440-kr-m                    40
jne3560-kr-xl                   40
jne3405-kr-xl                   38
set268-kr-np-xxl                38
set110-kr-pp-s                  38
jne3405-kr-l                    38
kurti           

In [52]:
pd.set_option('display.max_rows',20)

df2.head(14)

,date,months,customer,style,sku,size,pcs,rate,gross_amt,skuuuu
0,06-05-21,Jun-21,revathy loganathan,men5004,men5004-kr-l,l,1.0,616.56,617.0,l
1,06-05-21,Jun-21,revathy loganathan,men5004,men5004-kr-xl,xl,1.0,616.56,617.0,xl
2,06-05-21,Jun-21,revathy loganathan,men5004,men5004-kr-xxl,xxl,1.0,616.56,617.0,xxl
3,06-05-21,Jun-21,revathy loganathan,men5009,men5009-kr-l,l,1.0,616.56,617.0,l
4,06-05-21,Jun-21,revathy loganathan,men5011,men5011-kr-l,l,1.0,616.56,617.0,l
5,06-05-21,Jun-21,revathy loganathan,men5025,men5025-kr-l,l,1.0,649.03,649.0,l
6,06-05-21,Jun-21,revathy loganathan,men5015,men5015-kr-xl,xl,1.0,616.56,617.0,xl
7,06-05-21,Jun-21,revathy loganathan,men5022,men5022-kr-xxl,xxl,1.0,649.03,649.0,xxl
8,06-05-21,Jun-21,revathy loganathan,men5014,men5014-kr-s,s,1.0,649.03,649.0,s
9,06-05-21,Jun-21,revathy loganathan,men5013,men5013-kr-s,s,1.0,649.03,649.0,s


In [56]:
pd.set_option('display.max_rows', None)


# fill 'nan' sku in df3 with 'style-un-size' format
for idx, row in df3.iterrows():
    if str(row['sku']).strip().lower() == 'nan':
        style_val = str(row['style']).strip().lower()
        size_val = str(row['size']).strip().lower()
        df3.at[idx, 'sku'] = f"{style_val}-un-{size_val}"
df3['sku'].value_counts()

sku
shipping                      508
set268-kr-np-l                 62
jne3555-kr-l                   58
jne3548-kr-m                   54
jne3440-kr-l                   52
jne3548-kr-xl                  50
set268-kr-np-m                 50
jne3555-kr-m                   50
jne3555-kr-xl                  48
set268-kr-np-xl                46
set252-kr-pp-s.                46
jne3405-kr-m                   46
jne3555-kr-xxl                 44
jne2100-kr-144-m               44
jne3489-kr-xl                  42
jne3548-kr-l                   42
jne3440-kr-xl                  42
jne3489-kr-m                   42
jne3440-kr-m                   40
jne3567-kr-m                   40
jne3555-kr-s.                  40
set286-kr-np-xl                40
jne3560-kr-xl                  40
set268-kr-np-xxl               38
jne3548-kr-s                   38
jne3405-kr-l                   38
set110-kr-pp-s                 38
jne3440-kr-xxl                 38
jne3405-kr-xl                  38
kurti     

replaced all null values in sku with 'style-un-size' format where un stands for unknown

## Feature Engineering

#### Steps Here:

- drop dates because it has non-date types that can not be converted to date
- from months create two new features month and year and make them data type: datetime
- drop months
- use the month to create a new feature called season ie summer, winter, autum, and sprint
- drop skuuuu since its work is done

In [ ]:
 # 'L', 'XL', 'XXL', 'S', 'M', 'KURTI', 'XS', 'XXXL', 'XS.', 'M.', 'L.', 'S.' '5XL', '6XL', '4XL', 'S TO XXL',